# Intro to Bayesian Statistics Lab

Complete the following set of exercises to solidify your knowledge of Bayesian statistics and Bayesian data analysis.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## 1. Cookie Problem

Suppose we have two bowls of cookies. Bowl 1 contains 30 vanilla cookies and 10 chocolate cookies. Bowl 2 contains 20 of each. You randomly pick one cookie out of one of the bowls, and it is vanilla. Use Bayes Theorem to calculate the probability that the vanilla cookie you picked came from Bowl 1?

In [6]:
def bayes_theorem(priors: list, likelihoods: list) -> np.array:
    priors_x_likelihoods = np.multiply(priors, likelihoods)
    marginal_prob = sum(priors_x_likelihoods)
    return priors_x_likelihoods / marginal_prob #posterior probabilities

bowls = [1/2, 1/2] #Probability of selecting each jar
vanilla = [30/40, 10/20] #Probability of selecting a green of each jar

print(str(bayes_theorem(bowls, vanilla)[0]*100) + '%')

60.0%


What is the probability that it came from Bowl 2?

In [7]:
print(str(bayes_theorem(bowls, vanilla)[1]*100) + '%')

40.0%


What if the cookie you had picked was chocolate? What are the probabilities that the chocolate cookie came from Bowl 1 and Bowl 2 respectively?

In [19]:
chocolate = [1 - x for x in vanilla]

print(str(round(bayes_theorem(bowls, chocolate)[0]*100, 2)) + '%')

33.33%


In [20]:
print(str(round(bayes_theorem(bowls, chocolate)[1]*100,2)) + '%')

66.67%


## 2. Candy Problem

Suppose you have two bags of candies:

- In Bag 1, the mix of colors is:
    - Brown - 30%
    - Yellow - 20%
    - Red - 20%
    - Green - 10%
    - Orange - 10%
    - Tan - 10%
    
- In Bag 2, the mix of colors is:
    - Blue - 24%
    - Green - 20%
    - Orange - 16%
    - Yellow - 14%
    - Red - 13%
    - Brown - 13%
    
Not knowing which bag is which, you randomly draw one candy from each bag. One is yellow and one is green. What is the probability that the yellow one came from the Bag 1?

*Hint: For the likelihoods, you will need to multiply the probabilities of drawing yellow from one bag and green from the other bag and vice versa.*

In [18]:
bags = [1/2, 1/2]
yellow = [.2, .14]

print(str(round(bayes_theorem(bags, yellow)[0]*100,2)) + '%')

58.82%


What is the probability that the yellow candy came from Bag 2?

In [22]:
print(str(round(bayes_theorem(bags, yellow)[1]*100,2)) + '%')

41.18%


What are the probabilities that the green one came from Bag 1 and Bag 2 respectively?

In [23]:
green = [.1, .2]

print(str(round(bayes_theorem(bags, green)[0]*100,2)) + '%')

33.33%


In [24]:
print(str(round(bayes_theorem(bags, green)[1]*100,2)) + '%')

66.67%


## 3. Monty Hall Problem

Suppose you are a contestant on the popular game show *Let's Make a Deal*. The host of the show (Monty Hall) presents you with three doors - Door A, Door B, and Door C. He tells you that there is a sports car behind one of them and if you choose the correct one, you win the car!

You select Door A, but then Monty makes things a little more interesting. He opens Door B to reveal that there is no sports car behind it and asks you if you would like to stick with your choice of Door A or switch your choice to Door C. Given this new information, what are the probabilities of you winning the car if you stick with Door A versus if you switch to Door C?

In [39]:
door = [1/3, 1/3, 1/3]
car = [1/3, 1/3, 1/3]

bayes_theorem(door, car)

array([0.33333333, 0.33333333, 0.33333333])

In [34]:
# The probability of choosing to stick the door you chose is still 33.3%, 
# but the probability of it being behind any of the other two doors is 66.7%
# thus it is in our best interest to switch door

## 4. Bayesian Analysis 

Suppose you work for a landscaping company, and they want to advertise their service online. They create an ad and sit back waiting for the money to roll in. On the first day, the ad sends 100 visitors to the site and 14 of them sign up for landscaping services. Create a generative model to come up with the posterior distribution and produce a visualization of what the posterior distribution would look like given the observed data.

In [35]:
n = 100 #Total observations
k = 14 #Successful observations

def posterior_dist(k, n, size = 100000):
    prior = pd.Series(np.random.uniform(0, 1, size = size))
    outcomes = [np.random.binomial(n, p) for p in prior]
    return prior[[x == k for x in outcomes]]

posterior = posterior_dist(k, n)

Produce a set of descriptive statistics for the posterior distribution.

In [36]:
posterior.describe()

count    1046.000000
mean        0.147253
std         0.034822
min         0.058874
25%         0.121854
50%         0.145724
75%         0.169106
max         0.295941
dtype: float64

What is the 90% credible interval range?

In [37]:
from scipy.stats import sem, t

def conf_interval(sample, confidence):
    return t.interval(
        confidence, #confidence level
        len(sample) - 1, #degree of freedom
        loc = sample.mean(),
        scale = sem(sample)
    )

conf_interval(posterior, .90)

(0.1454801568981287, 0.14902530480029563)

What is the Maximum Likelihood Estimate?

In [38]:
def mle(arr): 
    rounded = arr.round(2)
    mode = rounded.mode()[0]
    prob = list(rounded).count(mode) / len(rounded)
    return mode, prob

mle(posterior)

(0.14, 0.12045889101338432)